In [100]:
from sklearn.model_selection import train_test_split
import pandas as pd
import lightgbm as lgb

In [101]:
import re

In [102]:
from sklearn.metrics import mean_absolute_error


In [103]:
df_NTC = pd.read_csv("../../data/NTC.csv")
df_losses = pd.read_csv("../../data/Avtice-losses.csv")

df_renewable = pd.read_csv("../../data/Forecast-renewable-generation.csv")
df_temp = pd.read_csv("../../data/Forecast-temperature.csv")

df_losses = df_losses.drop(df_losses.index[0])

In [104]:
#print(df_NTC.columns)
print(df_losses.columns)
#print(df_renewable.columns)
print(df_temp.columns)

Index(['Unnamed: 0', 'Wirkverluste/Active Losses'], dtype='object')
Index(['datetime', 'temperature_fore_ch', 'temperature_fore_fr',
       'temperature_fore_de', 'temperature_fore_it'],
      dtype='object')


In [105]:
df_losses = df_losses.rename(columns={
    'Unnamed: 0': 'datetime',
})

In [106]:

df_NTC['datetime'] = pd.to_datetime(df_NTC['datetime'])
df_losses['datetime'] = pd.to_datetime(df_losses['datetime'])
df_renewable['datetime'] = pd.to_datetime(df_renewable['datetime'])
df_temp['datetime'] = pd.to_datetime(df_temp['datetime'])

In [107]:
df_losses['Wirkverluste/Active Losses'] = pd.to_numeric(df_losses['Wirkverluste/Active Losses'], errors='coerce')

In [108]:
# Convert "Zeitstempel" column to datetime format, if not already
df_losses['datetime'] = pd.to_datetime(df_losses['datetime'])

# Subtract 15 minutes from each date in the "Zeitstempel" column
df_losses['datetime'] = df_losses['datetime'] - pd.Timedelta(minutes=15)



In [109]:

df_losses_new = df_losses.set_index('datetime')

hourly_loss = df_losses_new.resample('1H', closed='left', label='left').sum()

In [110]:
hourly_loss["Wirkverluste/Active Losses"]=hourly_loss["Wirkverluste/Active Losses"]/1000


In [111]:
#hourly_loss

In [112]:


# Check if the data is already in 6-hour intervals
if (df_temp['datetime'].diff().dt.total_seconds() / 3600).median() == 6:
    # If the data is in 6-hour intervals, leave the datetime column as it is
    hourly_temp = df_temp
else:
    # If the data is not in 6-hour intervals, resample it to hourly intervals
    hourly_temp = df_temp.resample('H', on='datetime').mean().reset_index()

    # Fill missing values in other columns with the values from the initial 6-hourly data
    columns_to_fill = hourly_temp.columns.difference(['datetime'])
    hourly_temp[columns_to_fill] = hourly_temp[columns_to_fill].fillna(method='ffill', limit=6)
    # Replace '6' with the appropriate number if you want to fill more or fewer hours

# Print the resulting hourly data
display(hourly_temp.head(10))

,datetime,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it
0,2019-01-01 01:00:00,4.1067,5.9729,7.4268,4.0281
1,2019-01-01 02:00:00,4.1067,5.9729,7.4268,4.0281
2,2019-01-01 03:00:00,4.1067,5.9729,7.4268,4.0281
3,2019-01-01 04:00:00,4.1067,5.9729,7.4268,4.0281
4,2019-01-01 05:00:00,4.1067,5.9729,7.4268,4.0281
5,2019-01-01 06:00:00,4.1067,5.9729,7.4268,4.0281
6,2019-01-01 07:00:00,1.7595,5.5398,6.3425,2.7281
7,2019-01-01 08:00:00,1.7595,5.5398,6.3425,2.7281
8,2019-01-01 09:00:00,1.7595,5.5398,6.3425,2.7281
9,2019-01-01 10:00:00,1.7595,5.5398,6.3425,2.7281


In [113]:
print(df_NTC.columns)
print(hourly_loss.columns)
print(df_renewable.columns)
print(hourly_temp.columns)

Index(['datetime', 'CH_AT', 'CH_DE', 'CH_FR', 'CH_IT', 'AT_CH', 'DE_CH',
       'FR_CH', 'IT_CH'],
      dtype='object')
Index(['Wirkverluste/Active Losses'], dtype='object')
Index(['datetime', 'solar_fore_de [MW]', 'solar_fore_it [MW]',
       'wind_fore_de [MW]', 'wind_fore_it [MW]'],
      dtype='object')
Index(['datetime', 'temperature_fore_ch', 'temperature_fore_fr',
       'temperature_fore_de', 'temperature_fore_it'],
      dtype='object')


In [114]:
hourly_loss = hourly_loss.reset_index()

In [115]:
print(hourly_loss)

                 datetime  Wirkverluste/Active Losses
0     2019-01-01 00:00:00                  139.525004
1     2019-01-01 01:00:00                  129.716036
2     2019-01-01 02:00:00                  133.398074
3     2019-01-01 03:00:00                  135.133852
4     2019-01-01 04:00:00                  131.699424
...                   ...                         ...
26299 2021-12-31 19:00:00                  171.707318
26300 2021-12-31 20:00:00                  159.462903
26301 2021-12-31 21:00:00                  155.109520
26302 2021-12-31 22:00:00                  171.370277
26303 2021-12-31 23:00:00                  146.054791

[26304 rows x 2 columns]


In [116]:
merged_df = df_NTC.merge(hourly_loss, on='datetime', how='outer')
merged_df = merged_df.merge(df_renewable, on='datetime', how='outer')
merged_df = merged_df.merge(hourly_temp, on='datetime', how='outer')

In [117]:
na_counts_per_column = merged_df.isna().sum()

In [118]:
na_counts_per_column 

datetime                         0
CH_AT                            3
CH_DE                            3
CH_FR                            3
CH_IT                            3
AT_CH                            3
DE_CH                            3
FR_CH                            3
IT_CH                            3
Wirkverluste/Active Losses       0
solar_fore_de [MW]               3
solar_fore_it [MW]               3
wind_fore_de [MW]                3
wind_fore_it [MW]                3
temperature_fore_ch           1578
temperature_fore_fr           1578
temperature_fore_de           1578
temperature_fore_it           1578
dtype: int64

In [119]:
merged_df_incomplete = merged_df.dropna()

In [120]:
merged_df_incomplete.columns

Index(['datetime', 'CH_AT', 'CH_DE', 'CH_FR', 'CH_IT', 'AT_CH', 'DE_CH',
       'FR_CH', 'IT_CH', 'Wirkverluste/Active Losses', 'solar_fore_de [MW]',
       'solar_fore_it [MW]', 'wind_fore_de [MW]', 'wind_fore_it [MW]',
       'temperature_fore_ch', 'temperature_fore_fr', 'temperature_fore_de',
       'temperature_fore_it'],
      dtype='object')

In [121]:
#merged_df_incomplete.reset_index(inplace=True)
#merged_df_incomplete=merged_df_incomplete.drop('datetime')

In [122]:
merged_df_incomplete=merged_df_incomplete.rename(columns={'Wirkverluste/Active Losses': 'Loss'})

In [123]:
merged_df_incomplete=merged_df_incomplete.rename(columns={'solar_fore_de [MW]': 'solar_fore_de'})
merged_df_incomplete=merged_df_incomplete.rename(columns={'solar_fore_it [MW]': 'solar_fore_it'})
merged_df_incomplete=merged_df_incomplete.rename(columns={'wind_fore_de [MW]': 'wind_fore_de'})
merged_df_incomplete=merged_df_incomplete.rename(columns={'wind_fore_it [MW]': 'wind_fore_it'})
#drop the datetime

#merged_df_incomplete=merged_df_incomplete.drop('datetime')


In [124]:
merged_df_incomplete = merged_df_incomplete.reset_index()

In [125]:
merged_df_incomplete = merged_df_incomplete.drop(columns=['datetime'])


In [126]:
merged_df_incomplete

,index,CH_AT,CH_DE,CH_FR,CH_IT,AT_CH,DE_CH,FR_CH,IT_CH,Loss,solar_fore_de,solar_fore_it,wind_fore_de,wind_fore_it,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it
0,1,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,129.716036,0.0,0.0,23052.3310,4596.5916,4.1067,5.9729,7.4268,4.0281
1,2,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,133.398074,0.0,0.0,24969.9701,4478.5564,4.1067,5.9729,7.4268,4.0281
2,3,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,135.133852,0.0,0.0,27082.9626,4323.3712,4.1067,5.9729,7.4268,4.0281
3,4,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,131.699424,0.0,0.0,26890.9717,4231.8283,4.1067,5.9729,7.4268,4.0281
4,5,700.0,4000.0,1200.0,2513.0,1200.0,800.0,3000.0,1910.0,147.391128,0.0,0.0,27740.1555,4266.3082,4.1067,5.9729,7.4268,4.0281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24727,26305,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,171.707318,0.0,0.0,36997.7200,1108.4000,8.5300,9.9600,10.7900,9.5900
24728,26306,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,159.462903,0.0,0.0,35666.9300,1077.9700,8.0000,9.4000,10.6300,9.1100
24729,26307,1200.0,4000.0,1400.0,3953.0,1200.0,800.0,3200.0,1810.0,155.109520,0.0,0.0,34383.8800,1048.2800,7.5000,8.8800,10.5100,8.6700
24730,26308,1200.0,4000.0,1400.0,3780.0,1200.0,800.0,3200.0,1810.0,171.370277,0.0,0.0,33075.2500,1078.7800,6.9700,8.5100,10.3200,8.1400


## Create 24 different dataframes, each for a time of the day

#### create all the lags

In [127]:
merged_ready_1= merged_df_incomplete

In [128]:
features = ['CH_DE', 'CH_FR', 'CH_IT', 'AT_CH', 'DE_CH', 'FR_CH', 'IT_CH',
               'solar_fore_de', 'solar_fore_it', 'wind_fore_de', 'wind_fore_it',
               'temperature_fore_ch', 'temperature_fore_fr', 'temperature_fore_de',
               'temperature_fore_it']

    # Create the lagged features
for feature in features:
    for lag in range(1, 336):
        col_name = f"{feature}_lag{lag}"
        merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)

merged_ready_1 = merged_ready_1.iloc[167:, :]

/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

...

/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_ready_1[col_name] = merged_ready_1[feature].shift(lag)
/tmp/ipykernel_60734/1983647430.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(a

In [129]:
merged_ready_1

,index,CH_AT,CH_DE,CH_FR,CH_IT,AT_CH,DE_CH,FR_CH,IT_CH,Loss,...,temperature_fore_it_lag326,temperature_fore_it_lag327,temperature_fore_it_lag328,temperature_fore_it_lag329,temperature_fore_it_lag330,temperature_fore_it_lag331,temperature_fore_it_lag332,temperature_fore_it_lag333,temperature_fore_it_lag334,temperature_fore_it_lag335
167,168,700.0,4000.0,1200.0,1074.0,1200.0,800.0,3000.0,1910.0,142.911661,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,169,700.0,4000.0,1200.0,1074.0,1200.0,800.0,3000.0,1910.0,156.728192,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,170,700.0,4000.0,1200.0,1074.0,1200.0,800.0,3000.0,1910.0,166.503696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,171,700.0,4000.0,1200.0,1097.0,1200.0,800.0,3000.0,1910.0,185.935455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,172,700.0,4000.0,1200.0,1097.0,1200.0,800.0,3000.0,1910.0,185.184283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24727,26305,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,171.707318,...,4.37,4.50,4.54,4.53,4.66,4.96,5.17,5.43,6.11,6.54
24728,26306,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,159.462903,...,4.19,4.37,4.50,4.54,4.53,4.66,4.96,5.17,5.43,6.11
24729,26307,1200.0,4000.0,1400.0,3953.0,1200.0,800.0,3200.0,1810.0,155.109520,...,4.10,4.19,4.37,4.50,4.54,4.53,4.66,4.96,5.17,5.43
24730,26308,1200.0,4000.0,1400.0,3780.0,1200.0,800.0,3200.0,1810.0,171.370277,...,4.61,4.10,4.19,4.37,4.50,4.54,4.53,4.66,4.96,5.17


#### create the different df

In [130]:
dfs_total = {}

# Create 24 copies of the original dataframe
for i in range(1, 49):
    working_df = merged_ready_1.copy()
    
    threshold = i-1

    # Extract columns to drop based on their name
    cols_to_drop = []
    for col in working_df.columns:
        match = re.search(r'lag(\d+)', col)
        if match:
            lag_number = int(match.group(1))
            if lag_number > threshold:
                cols_to_drop.append(col)

    # Drop these columns
    working_df = working_df.drop(columns=cols_to_drop)
    
    
    
    dfs_total[f'df_{i}'] = working_df



In [132]:
dfs_total['df_46']

,index,CH_AT,CH_DE,CH_FR,CH_IT,AT_CH,DE_CH,FR_CH,IT_CH,Loss,...,temperature_fore_it_lag36,temperature_fore_it_lag37,temperature_fore_it_lag38,temperature_fore_it_lag39,temperature_fore_it_lag40,temperature_fore_it_lag41,temperature_fore_it_lag42,temperature_fore_it_lag43,temperature_fore_it_lag44,temperature_fore_it_lag45
167,168,700.0,4000.0,1200.0,1074.0,1200.0,800.0,3000.0,1910.0,142.911661,...,2.9418,2.9418,2.9418,2.9418,2.9418,2.9418,0.9218,0.9218,0.9218,0.9218
168,169,700.0,4000.0,1200.0,1074.0,1200.0,800.0,3000.0,1910.0,156.728192,...,11.6246,2.9418,2.9418,2.9418,2.9418,2.9418,2.9418,0.9218,0.9218,0.9218
169,170,700.0,4000.0,1200.0,1074.0,1200.0,800.0,3000.0,1910.0,166.503696,...,11.6246,11.6246,2.9418,2.9418,2.9418,2.9418,2.9418,2.9418,0.9218,0.9218
170,171,700.0,4000.0,1200.0,1097.0,1200.0,800.0,3000.0,1910.0,185.935455,...,11.6246,11.6246,11.6246,2.9418,2.9418,2.9418,2.9418,2.9418,2.9418,0.9218
171,172,700.0,4000.0,1200.0,1097.0,1200.0,800.0,3000.0,1910.0,185.184283,...,11.6246,11.6246,11.6246,11.6246,2.9418,2.9418,2.9418,2.9418,2.9418,2.9418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24727,26305,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,171.707318,...,6.7200,6.7100,6.8800,7.0800,7.4300,7.7300,7.9400,8.0200,8.5700,8.9900
24728,26306,1200.0,4000.0,1400.0,4069.0,1200.0,800.0,3200.0,1810.0,159.462903,...,7.4200,6.7200,6.7100,6.8800,7.0800,7.4300,7.7300,7.9400,8.0200,8.5700
24729,26307,1200.0,4000.0,1400.0,3953.0,1200.0,800.0,3200.0,1810.0,155.109520,...,8.8600,7.4200,6.7200,6.7100,6.8800,7.0800,7.4300,7.7300,7.9400,8.0200
24730,26308,1200.0,4000.0,1400.0,3780.0,1200.0,800.0,3200.0,1810.0,171.370277,...,10.3800,8.8600,7.4200,6.7200,6.7100,6.8800,7.0800,7.4300,7.7300,7.9400


In [103]:


def compute_mae(merged_df_ready_prep, lag_input):
    # Ensure the merged_ready DataFrame is created first
    merged_ready = merged_df_ready_prep.copy()

    

    X = merged_ready.drop('Loss', axis=1)  # Features
    y = merged_ready['Loss']  # Target variable

    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'mae',
        'device': 'gpu',  # Enable GPU support
        #'gpu_platform_id' = 0,
        'gpu_device_id' = 1
    }

    # Create the LightGBM dataset
    train_data = lgb.Dataset(X_train, label=y_train)
    
    # Train the model
    model = lgb.train(params, train_data)

    # Predict the test data
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    # Calculate MAE
    mae = mean_absolute_error(y_test, y_pred)

    return mae



SyntaxError: invalid syntax (3034853253.py, line 30)

In [ ]:


# Values for i and j
i_values = list(range(1, 3))
j_values = list(range(1, 25))

# Creating the DataFrame
data_results_hope = {}
for i in i_values:
    col_data = []
    for j in j_values:
        col_data.append(compute_mae(i, j))
    data_results_hope[i] = col_data

df = pd.DataFrame(data, index=j_values)

print(df)


In [133]:
#lag_1=compute_mae(merged_ready, 1)

In [134]:
#lag_168=compute_mae(merged_ready, 168)


In [135]:

#lag_336=compute_mae(merged_ready, 336)




In [47]:
# to do:

In [49]:
# make the it for 
# do it for each day 24 forecasts


